In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import pylab as plt
import torch
import torch.nn
from scipy.spatial.transform import Rotation as R

from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix

from utils import utils
from config import config as cfg
from data_loader import get_dataloader
from models import VoxelNet

In [ ]:
score_threshold = 0.99
pretrained_model = 'model-new-iou.ckpt'
sample_token = '7eb3e546df5311b035f1d4b7e88351ffdd85f311ef1bdd22f71a160e92386fed'
device = torch.device('cuda:1')

In [ ]:
data_loaders, data_sizes = get_dataloader(phases=['test'])

In [ ]:
model = VoxelNet().to(device)
checkpoint = torch.load(os.path.join(cfg.work_dir, 'data/models/pretrain', pretrained_model))
model.load_state_dict(checkpoint['state_dict'])
model.eval()
print()

In [ ]:
for idx, (voxel_features, voxel_coords) in enumerate(data_loaders['test']):
    voxel_features = voxel_features.to(device)
    voxel_coords = voxel_coords.to(device)
    
    # psm: [batch_size, R_z, H_map, W_map]
    # rm: [batch_size, R_z * B_encode, H_map, W_map]
    psm, rm = model(voxel_features, voxel_coords, device)
    batch_size = psm.size(0)

    # psm: [batch_size, H_map, W_map, R_z]
    psm = torch.sigmoid(psm.permute(0,2,3,1))

    # psm: [batch_size, H_map * W_map * R_z]
    psm = psm.reshape((cfg.batch_size, -1))

    # rm: [batch_size, H_map, W_map, R_z * B_encode]
    rm = rm.permute(0,2,3,1).contiguous()

    # rm: [batch_size, H_map, W_map, R_z, R_z * B_encode]
    rm = rm.view(rm.size(0), rm.size(1), rm.size(2), 14)

    # prob: [batch_size, H_map * W_map * R_z]
    prob = psm.view(batch_size, -1)
    
    # batch_boxes3d: [batch_size, H_map * W_map * R_z, B_encode]
    batch_boxes3d = utils.delta_to_boxes3d(rm, device)

    mask = torch.gt(prob, score_threshold)

    mask_reg = mask.unsqueeze(2).repeat(1, 1, 7)

    batch_id = 0

    # boxes3d: [H_map * W_map * R_z, B_encode]
    boxes3d = torch.masked_select(batch_boxes3d[batch_id], mask_reg[batch_id]).view(-1, 7)
    scores = torch.masked_select(prob[batch_id], mask[batch_id])

    # boxes2d_corners: [H_map * W_map * R_z, 4 corners, 2 xy]
    boxes3d = boxes3d.cpu().detach().numpy()
    boxes2d_corners = utils.boxes3d_to_corners(boxes3d)

    print("Predicted 2-D Anchor Boxes:", boxes2d_corners.shape)
    
    break

In [ ]:
fig, ax = plt.subplots(figsize=(40, 40))

ax.plot(boxes2d_corners[:, :, 0].astype(np.float32), boxes2d_corners[:, :, 1].astype(np.float32),
        'o', markersize=10)

ax.set_xlim([-80, 80])
ax.set_ylim([-80, 80])

In [ ]:
lyft_dataset = data_loaders['test'].dataset.lyft_dataset
sample = lyft_dataset.get('sample', sample_token)
lidar_info = lyft_dataset.get('sample_data', sample['data']['LIDAR_TOP'])
lidar_data_path = lyft_dataset.get_sample_data_path(sample['data']['LIDAR_TOP'])
gt_boxes3d = lyft_dataset.get_boxes(sample['data']['LIDAR_TOP'])

ego_pose = lyft_dataset.get('ego_pose', lidar_info['ego_pose_token'])
calibrated_sensor = lyft_dataset.get('calibrated_sensor', lidar_info['calibrated_sensor_token'])

pointclouds = LidarPointCloud.from_file(lidar_data_path)
pointclouds = pointclouds.points.transpose(1, 0)

In [ ]:
fig, ax = plt.subplots(figsize=(40, 40))

ax.plot(pointclouds[:, 0], pointclouds[:, 1], 'o')
ax.plot(boxes2d_corners[:, :, 0].astype(np.float32), boxes2d_corners[:, :, 1].astype(np.float32),
        'o', markersize=10)

ax.set_xlim([-80, 80])
ax.set_ylim([-80, 80])